# 임포트

In [18]:
# import torch as t
# import urllib.request
# import pandas as pd

# hugging Face의 한국어 프리트레인 된 Llama

1. tokenizer, model 불러오고 정의하기
2. 보낼 메세지와 행동 정의하기
3. tokenizer를 통해 인풋할 메세지를 변환해주고 input_ids 변수에 넣기
4. terminators로 마지막 문자 정의하기
5. model.을 활용하여 응답 생성
6. response 받기

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/Llama-3-Open-Ko-8B")
model = AutoModelForCausalLM.from_pretrained("beomi/Llama-3-Open-Ko-8B")

# # 또는

# model_id = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype="auto", # 적절한 데이터 타입과
#     device_map="auto", # 장치를 선택하도록 하는 코드 예) GPU
# )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# 챗봇에게 보낼 메시지와 행동

messages = [
    {"role": "system", "content": "친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
    {"role": "user", "content": "피보나치 수열이 뭐야? 그리고 피보나치 수열에 대해 파이썬 코드를 짜줘볼래?"},
]

In [ ]:
# 메시지를 템플릿에 맞게 변환하고 토크나이저를 통해 텐서로 변환

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [ ]:
# 종료 토큰을 설정
# 빈 문자열을 토큰 ID로 변환하여 종료 조건을 설정한다

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
# 응답 생성

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1,
    top_p=0.9,
)

In [ ]:
# 응답 디코딩 출력

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

# 로컬 다운로드 후 사용하는 기본 llama3

Langchain을 통해 접근하는 방법은 별도의 tokenizer나 terminators가 필요하지 않음
1. Langchain을 통해 임포트
2. 모델 인스턴스 생성
3. 응답 생성 함수 정의
4. response 받기

In [2]:
from langchain_community.llms import Ollama
from langchain import PromptTemplate

llm = Ollama(model="llama3", stop=["<|eot_id|>"])

In [3]:

def get_model_response(user_prompt, system_prompt):
  template = """
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    {system_prompt}
    <|start_header_id|>user<|end_header_id|>
    {user_prompt}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id
  """
  prompt = PromptTemplate(
      template=template,
      input_variables=["user_prompt", "system_prompt"]
  )

  response = llm(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))
  return response

In [ ]:
get_model_response("당신은 사랑받기 위해 태어난 사람", "노래가사 작성해주세요. 한국어로")

# 한국어로 프리트레인 된 Llama

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d"
)

response = client.chat.completions.create(
    model="xionic-ko-llama-3-70b",
    messages=[
        {"role": "system", "content": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer in korean."},
        {"role": "user", "content": "각국 의 법률에서는 정의라는 개념이 자주 등장하며, 법령의 형성과 해석에 있어 매우 중요한 부분을 차지한다. 하지만 정의란 명>확히 규정 할 수 없는 개념이기에 해석의 논란 이 있을 수 있다. 그렇다면 사회구성원의 대다수가 납득할 수 있는 보편적 정의를 입증하는 방법은 무엇일지 생각해보아라."}
    ]
)

print (response)

ChatCompletion(id='cmpl-c4cc9f85600b4378abf4215e3fdc8d7b', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='정의라는 개념은 법률에서 매우 중요한 역할을 하며, 법령의 형성과 해석에 있어 필수적인 요소로 간주된다. 그러나 정의는 명확히 규정할 수 없는 개념이므로, 이를 둘러싼 해석의 논란이 항상 존재할 수 있다. 이러한 문제를 해결하기 위해, 사회구성원의 대다수가 납득할 수 있는 보편적 정의를 입증하는 방법을 찾아야 할 것이다. 이러한 방법을 찾기 위해, 우선 정의의 개념을 살펴보아야 하며, 이를 뒷받침하는 이론적 근거를 찾아야 한다. 또한, 정의의 개념을 실제 사회에서 적용하는 방법을 찾아야 하며, 이를 통해 사회구성원의 대다수가 납득할 수 있는 보편적 정의를 입증할 수 있는지 확인해야 한다. 이를 위해, 본 글에서는 정의의 개념을 살펴보고, 이를 뒷받침하는 이론적 근거를 찾아, 실제 사회에서 적용하는 방법을 찾아보아, 사회구성원의 대다수가 납득할 수 있는 보편적 정의를 입증하는 방법을 제시하고자 한다.', role='assistant', function_call=None, tool_calls=None), stop_reason=None)], created=1721192982, model='xionic-ko-llama-3-70b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=272, prompt_tokens=136, total_tokens=408))
